In [31]:
from fishsense_api_sdk.client import Client
from tqdm.asyncio import tqdm_asyncio
from tqdm.notebook import tqdm
import numpy as np

In [2]:
URL = "http://localhost:8000"

In [4]:
async with Client(URL) as client:
    dives = await client.dives.get()
    dives = [dive for dive in dives if dive.priority == "HIGH"]

len(dives), dives[0]

(7,
 Dive(id=279, name='111323_Alligator DeepEast Drift2_FSL05', path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05', dive_datetime=datetime.datetime(2023, 11, 13, 12, 34, 8, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=5, dive_slate_id=7))

In [10]:
async with Client(URL) as client:
    species_label_list = await tqdm_asyncio.gather(*[client.labels.get_species_labels(dive.id) for dive in dives])

len(species_label_list), species_label_list[0]

100%|██████████| 7/7 [00:01<00:00,  4.83it/s]


(7,
 [SpeciesLabel(id=1024, label_studio_task_id=222252, label_studio_project_id=57, image_url='https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/7252dfb0db0b855616e45607f58c1c93', updated_at=datetime.datetime(2025, 10, 20, 20, 42, 14, 832143, tzinfo=TzInfo(0)), completed=True, grouping=None, top_three_photos_of_group=None, slate_upside_down=True, laser_x=1970.0, laser_y=1340.0, laser_label='Red Laser', content_of_image='Slate, Laser not on slate', fish_measurable_category=None, fish_angle_category=None, fish_curved_category=None, label_studio_json={'annotations': [{'id': 117942, 'result': [{'id': 'result1', 'type': 'keypointlabels', 'value': {'x': 49.08221189998697, 'y': 44.44128413294742, 'width': 0.2, 'keypointlabels': ['Red Laser']}, 'origin': 'prediction-changed', 'to_name': 'image', 'from_name': 'laser', 'image_rotation': 0, 'original_width': 4014, 'original_height': 3016}, {'id': 'm4K68yydiZ', 'type': 'taxonomy', 'value': {'taxonomy': [['Slate', 'Laser not on s

In [32]:
def image_coordinate_to_projected_point(
    image_point: np.ndarray, inverted_camera_matrix: np.ndarray
) -> np.ndarray:
    assert isinstance(image_point, np.ndarray)
    assert isinstance(inverted_camera_matrix, np.ndarray)

    homogenous_point_image_space = np.zeros(3, dtype=float)
    homogenous_point_image_space[:2] = image_point
    homogenous_point_image_space[2] = 1

    # define laser ray assuming pinhole camera
    return inverted_camera_matrix @ homogenous_point_image_space

In [33]:
def compute_world_point(
    laser_origin: np.ndarray,
    laser_axis: np.ndarray,
    inverted_camera_matrix: np.ndarray,
    image_coordinate: np.ndarray,
) -> np.ndarray:
    projected_point = image_coordinate_to_projected_point(
        image_point=image_coordinate, inverted_camera_matrix=inverted_camera_matrix
    )
    final_laser_axis = -1 * projected_point / np.linalg.norm(projected_point)

    # point_constant = (-1 * laser_axis[2] * laser_origin[0]) / \
    #     (laser_axis[0] * final_laser_axis[2] - laser_axis[2] * final_laser_axis[0])

    # Least squares
    point_constant = (
        (final_laser_axis.T @ laser_origin)
        - ((laser_axis.T @ laser_origin) * (laser_axis.T @ final_laser_axis))
    ) / (1 - (laser_axis.T @ final_laser_axis) ** 2)

    world_point = point_constant * final_laser_axis
    return world_point

In [38]:
async with Client(URL) as client:
    for species_labels, dive in tqdm(list(zip(species_label_list, dives))):
        camera = await client.cameras.get(dive.camera_id)
        camera_intrinsics = await client.cameras.get_intrinsics(camera.id)
        laser_extrinsics = await client.dives.get_laser_extrinsics(dive.id)

        filtered_species_labels = [label for label in species_labels if label.top_three_photos_of_group]

        laser_labels = await tqdm_asyncio.gather(*[client.labels.get_laser_label(label.image_id) for label in filtered_species_labels])
        headtail_labels = await tqdm_asyncio.gather(*[client.labels.get_headtail_label(label.image_id) for label in filtered_species_labels])

        species_label_by_image_id = {label.image_id: label for label in species_labels}
        laser_label_by_image_id = {label.image_id: label for label in laser_labels}
        headtail_label_by_image_id = {label.image_id: label for label in headtail_labels}

        for image_id in species_label_by_image_id.keys():
            laser_label = laser_label_by_image_id[image_id]
            # headtail_label = headtail_label_by_image_id[image_id]

            # laser_2d = np.array([laser_label.x, laser_label.y])
            # laser3d = compute_world_point(laser_origin=laser_extrinsics.position,
            #                               laser_axis=laser_extrinsics.direction,
            #                               inverted_camera_matrix=np.linalg.inv(camera_intrinsics.inverted_camera_matrix),
            #                               image_coordinate=laser_2d)

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 15/15 [00:00<00:00, 25.04it/s]


KeyError: 78666